In [1]:
# path to your train/test/meta folders
DATA_PATH = "."

# names of valuable files/folders
train_meta_fname = "train.csv"
test_meta_fname = "sample_submission.csv"
train_data_folder = "train"
test_data_folder = "test"

In [2]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchaudio
from efficientnet_pytorch import EfficientNet
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from tqdm import tqdm

In [3]:
# set seeds
import random
import numpy as np

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)
torch.backends.cudnn.deterministic = True

In [4]:
df_train = pd.read_csv(os.path.join(DATA_PATH, train_meta_fname))
df_test = pd.read_csv(os.path.join(DATA_PATH, test_meta_fname))
df_train.head(2)

,fname,label
0,8bcbcc394ba64fe85ed4.wav,Finger_snapping
1,00d77b917e241afa06f1.wav,Squeak


In [5]:
n_classes = df_train.label.nunique()
print(n_classes)
classes_dict = {cl: i for i, cl in enumerate(df_train.label.unique())}
df_train["label_encoded"] = df_train.label.map(classes_dict)
df_train.head()

41


,fname,label,label_encoded
0,8bcbcc394ba64fe85ed4.wav,Finger_snapping,0
1,00d77b917e241afa06f1.wav,Squeak,1
2,17bb93b73b8e79234cb3.wav,Electric_piano,2
3,7d5c7a40a936136da55e.wav,Harmonica,3
4,17e0ee7565a33d6c2326.wav,Snare_drum,4


In [6]:
class SoundModel(nn.Module):
    def __init__(self, sample_rate=16000, n_classes=41):
        super().__init__()
        self.ms = torchaudio.transforms.MelSpectrogram(
            sample_rate=sample_rate, hop_length=126
        )

        self.layers = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=10, kernel_size=(3, 3), padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=10, out_channels=3, kernel_size=(3, 3), padding=1),
            nn.ReLU(),
            EfficientNet.from_pretrained("efficientnet-b0"),
            nn.ReLU(),
            nn.Linear(1000, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Linear(128, n_classes),
        )

    def forward(self, x):
        return self.layers(self.ms(x))

    def inference(self, x):
        x = self.forward(x)
        x = F.softmax(x, dim=1)
        return x

In [7]:
def sample_or_pad(waveform, wav_len=32000):
    m, n = waveform.shape
    if n < wav_len:
        padded_wav = torch.zeros(1, wav_len)
        padded_wav[:, :n] = waveform
        return padded_wav
    elif n > wav_len:
        offset = np.random.randint(0, n - wav_len)
        sampled_wav = waveform[:, offset : offset + wav_len]
        return sampled_wav
    else:
        return waveform


class EventDetectionDataset(Dataset):
    def __init__(self, data_path, x, y=None):
        self.x = x
        self.y = y
        self.data_path = data_path

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        path2wav = os.path.join(self.data_path, self.x[idx])
        waveform, sample_rate = torchaudio.load(path2wav, normalize=True)
        waveform = sample_or_pad(waveform)
        if self.y is not None:
            return waveform, self.y[idx]
        return waveform

In [8]:
def eval_model(model, eval_dataset):
    model.eval()
    forecast, true_labs = [], []
    with torch.no_grad():
        for wavs, labs in tqdm(eval_dataset):
            wavs, labs = wavs.cuda(), labs.detach().numpy()
            true_labs.append(labs)
            outputs = model.inference(wavs)

            outputs = outputs.detach().cpu().numpy().argmax(axis=1)
            forecast.append(outputs)
    forecast = [x for sublist in forecast for x in sublist]
    true_labs = [x for sublist in true_labs for x in sublist]
    return f1_score(forecast, true_labs, average="macro")

In [9]:
n_epoch = 100
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
x_data = df_train.fname.values
y_data = df_train.label_encoded.values
for fold, (train_idx, val_idx) in enumerate(skf.split(x_data, y_data)):
    X_train = x_data[train_idx]
    y_train = y_data[train_idx]
    train_loader = DataLoader(
        EventDetectionDataset(
            os.path.join(DATA_PATH, train_data_folder), X_train, y_train
        ),
        batch_size=128,
        shuffle=True,
    )
    X_val = x_data[val_idx]
    y_val = y_data[val_idx]
    val_loader = DataLoader(
        EventDetectionDataset(os.path.join(DATA_PATH, train_data_folder), X_val, y_val),
        batch_size=128,
    )

    model = SoundModel()
    model = model.cuda()
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    scheduler = torch.optim.lr_scheduler.LambdaLR(
        optimizer, lr_lambda=lambda epoch: 0.95**epoch
    )

    best_f1 = 0
    for epoch in range(n_epoch):
        model.train()
        for wavs, labs in tqdm(train_loader):
            optimizer.zero_grad()
            wavs, labs = wavs.cuda(), labs.cuda()
            outputs = model(wavs)
            loss = criterion(outputs, labs)
            loss.backward()
            optimizer.step()
        scheduler.step()
        f1 = eval_model(model, val_loader)
        f1_train = eval_model(model, train_loader)
        print(f"epoch: {epoch}, f1_val: {f1}, f1_train: {f1_train}")
        if f1 > best_f1:
            best_f1 = f1
            torch.save(model.state_dict(), f"model_{fold}.ckpt")

    print("Best F1 on validation:", best_f1)

Loaded pretrained weights for efficientnet-b0


C:\ProgramData\Anaconda3\envs\nlp-env\lib\site-packages\torchaudio\functional\functional.py:584: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  warnings.warn(
C:\ProgramData\Anaconda3\envs\nlp-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 36/36 [00:04<00:00,  8.65it/s]


epoch: 0, f1_val: 0.04162398126790363, f1_train: 0.055729607465809475


100%|██████████| 36/36 [00:04<00:00,  8.66it/s]


epoch: 1, f1_val: 0.1606833804214681, f1_train: 0.15407633534469023


...

epoch: 97, f1_val: 0.6702654618667948, f1_train: 0.8998314730824739


100%|██████████| 36/36 [00:05<00:00,  7.13it/s]


epoch: 98, f1_val: 0.6568362932260938, f1_train: 0.8969336667305738


100%|██████████| 36/36 [00:05<00:00,  6.42it/s]

epoch: 99, f1_val: 0.6510310786518454, f1_train: 0.8912039216122086
Best F1 on validation: 0.6989217127835318


In [10]:
test_loader = DataLoader(
    EventDetectionDataset(
        os.path.join(DATA_PATH, test_data_folder), df_test.fname.values, None
    ),
    batch_size=128,
    shuffle=False,
)
models = [SoundModel().cuda() for _ in range(5)]
for i in range(5):
    models[i].load_state_dict(torch.load(f"model_{i}.ckpt"))
    models[i].eval()
forecast = []
with torch.no_grad():
    for wavs in tqdm(test_loader):
        wavs = wavs.cuda()
        outputs = models[0].inference(wavs)
        for i in range(1, 5):
            outputs += models[i].inference(wavs)
        outputs = outputs.detach().cpu().numpy().argmax(axis=1)
        forecast.append(outputs)
forecast = [x for sublist in forecast for x in sublist]
decoder = {classes_dict[cl]: cl for cl in classes_dict}
forecast = pd.Series(forecast).map(decoder)
df_test["label"] = forecast
df_test.to_csv(f"submission.csv", index=None)

C:\ProgramData\Anaconda3\envs\nlp-env\lib\site-packages\torchaudio\functional\functional.py:584: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  warnings.warn(


Loaded pretrained weights for efficientnet-b0
Loaded pretrained weights for efficientnet-b0
Loaded pretrained weights for efficientnet-b0
Loaded pretrained weights for efficientnet-b0
Loaded pretrained weights for efficientnet-b0


100%|██████████| 30/30 [00:12<00:00,  2.34it/s]
